### Format and combine frame txt files of yolo_traking 
path should be specified accordingly

#### MOT20

In [1]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = os.path.basename(file_path).split('_')[-1].split('.')[0]
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    files = [file_name for file_name in os.listdir(label_directory) if file_name.startswith(video_name) and file_name.endswith('.txt')]
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        #print(f"Processing file: {file_path}")
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    # trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    trackers = ['deepocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/MOT20/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/MOT20'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


Processing videos for deepocsort:   0%|          | 0/4 [00:00<?, ?video/s]

No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-02/labels/MOT20-02_1.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-02/labels/MOT20-02_2.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-02/labels/MOT20-02_3.txt


Processing videos for deepocsort:  25%|██▌       | 1/4 [00:00<00:01,  2.34video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/DEEPOCSORT__input_1280__conf_.25/data/MOT20-02.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-03/labels/MOT20-03_1.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-03/labels/MOT20-03_2.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-03/labels/MOT20-03_3.txt


Processing videos for deepocsort:  50%|█████     | 2/4 [00:01<00:01,  1.86video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/DEEPOCSORT__input_1280__conf_.25/data/MOT20-03.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-05/labels/MOT20-05_1.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-05/labels/MOT20-05_2.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-05/labels/MOT20-05_3.txt


Processing videos for deepocsort: 100%|██████████| 4/4 [00:01<00:00,  2.46video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/DEEPOCSORT__input_1280__conf_.25/data/MOT20-05.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-01/labels/MOT20-01_1.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-01/labels/MOT20-01_2.txt
No data processed for file: /workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20_.25__cmc_off/deepocsort__input_1280__conf_.25/MOT20-01/labels/MOT20-01_3.txt
Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/DEEPOCSORT__input_1280__conf_.25/data/MOT20-01.txt


#### PersonPath22

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0]) + 1
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['ocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/PersonPath22/test'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/PersonPath22_.25'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/person_path_22-test'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### VIRAT-S

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0]) + 1
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number - 1},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/VIRAT-S/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/VIRAT-S'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/virat_s-test'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


#### KITTI

In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0]) + 1
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                    
                    # Convert normalized coordinates to pixel values
                    x_center *= image_width
                    y_center *= image_height
                    width *= image_width
                    height *= image_height
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number - 1},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/KITTI/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/KITTI'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/KITTI'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


### Visualize Formated txt
path should be specified accordingly

#### MOT20

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/MOT20/train/MOT20-05/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/MOT20-train/DeepOC-SORT__input_1280__conf_.25/data/MOT20-05.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_MOT20/viz_deepocsort__input_1280__conf_.25/MOT20-05'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


#### PersonPath22

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/PersonPath22/test/uid_vid_00011.mp4/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/PersonPath22/BYTETRACK__input_1280__conf_.25/uid_vid_00011.mp4.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_PersonPath22/viz_bytetrack__input_1280_conf_.25/uid_vid_00011'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


In [ ]:
import os
import cv2

def create_video_from_images(image_dir, output_video_path, fps=30):
    # Get a list of all image files in the directory
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    image_files.sort()  # Ensure the files are sorted in the correct order

    # Read the first image to get the dimensions
    first_image_path = os.path.join(image_dir, image_files[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        frame = cv2.imread(image_path)
        video.write(frame)

    # Release the VideoWriter
    video.release()
    print(f"Video saved to {output_video_path}")

# Example usage
image_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_PersonPath/viz_bytetrack__input_1280_conf_.25/uid_vid_00008'  # Adjust this to your image directory
output_video_path = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_PersonPath/viz_bytetrack__input_1280_conf_.25/uid_vid_00008.mp4'  # Adjust this to your desired output video path

create_video_from_images(image_dir, output_video_path)


#### VIRAT-S

In [ ]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/VIRAT-S/train/VIRAT_S_010000_06_000728_000762/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/VIRAT-S/ByteTRACK__input_1280__conf_.25/data/VIRAT_S_010000_06_000728_000762.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_VIRAT-S/viz_ByteTRACK__input_1280__conf_.25/VIRAT_S_010000_06_000728_000762'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = os.path.basename(file_path).split('_')[-1].split('.')[0]
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    files = [file_name for file_name in os.listdir(label_directory) if file_name.startswith(video_name) and file_name.endswith('.txt')]
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        #print(f"Processing file: {file_path}")
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['ocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/MOT17/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT17_.25_3_hits'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/MOT17_.25_3_hits'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = os.path.basename(file_path).split('_')[-1].split('.')[0].lstrip('0')  # Remove leading zeros
    frame_number = frame_number if frame_number else '0'  # Ensure frame_number is not empty
    output_data = []
    print(f"Processing file: {file_path}")
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{int(frame_number)},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    print(f"Processed {len(output_data)} lines from file: {file_path}")
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory {label_directory} does not exist.")
        return

    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found for video {video_name} in directory {label_directory}.")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['ocsort']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25", "data")
        print(f"result_dir: {result_dir}")
        print(f"output_dir: {output_dir}")
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            if not os.path.exists(image_directory):
                print(f"Image directory {image_directory} does not exist.")
                continue
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/MOT17/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT17-train'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/MOT17-train'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)
